# [실습] Multilayer Perceptron(MLP)로 숫자 분류기 구현하기
- MLP는 Fully Connect layer, Dense layer 등의 이름으로 불림
- 기존 cnn에 많이 활용되는 구조
- hidden layer가 추가 + activation
- linear transformation, affine transform은 아무리 여러번 반복해도 affine이라 중간 중간에 non-lineard를 줘야 훨씬 더 복잡한 함수로 모델링할 수 있습니다
- 각 레이어의 output이 나오면 activation function을 통과함 (sigmoid, tanh, relu)
- 마지막 output이 나오면 softmax를 취해서 cross entropy에 넣으면 logistic regression
- 우리의 코드에선 softmax를 취하지 않을겁니다! 
- softmax를 취하기 전의 값을 logit이라 부르고, 이것을 입력으로 받는 텐서플로우 함수가 있는데 그것을 활용해보겠습니다

### Logistic regression과 Multi layer perceptron의 차이
- Depth!!!


- 아래 코드는 모델, Cost 정의 -> 초기화 -> 실행 순으로 진행

In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
%matplotlib inline  
print ("PACKAGES LOADED")

PACKAGES LOADED


In [3]:
mnist = input_data.read_data_sets('data/', one_hot=True)

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz


In [6]:
n_input    = 784 
n_hidden_1 = 256 
n_hidden_2 = 128 
n_hidden_3 = 64
n_classes  = 10  

x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])
    
# weight와 bias가 여러개 있고, dict으로 들어가있음
stddev = 0.1
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1], stddev=stddev)),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2], stddev=stddev)),
    'h3': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_3], stddev=stddev)),
    'out': tf.Variable(tf.random_normal([n_hidden_3, n_classes], stddev=stddev))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'b3': tf.Variable(tf.random_normal([n_hidden_3])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}
print ("NETWORK READY")

NETWORK READY


In [11]:
# model
def multilayer_preceptron(_X, _weights, _biases):
    layer_1 = tf.nn.relu(tf.add(tf.matmul(_X, _weights['h1']), _biases['b1']))
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, _weights['h2']), _biases['b2']))
    layer_3 = tf.nn.sigmoid(tf.add(tf.matmul(layer_2, _weights['h3']), _biases['b3']))
    return (tf.matmul(layer_3, _weights['out']) + _biases['out'])
                                                               
# 위 함수는 softmax가 없습니다!
# softmax_cross_entropy_with_logits을 활용할 예정이라 그렇습니다!

# PREDICTION
pred = multilayer_preceptron(x, weights, biases)

# LOSS AND OPTIMIZER
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=pred)) 
# 들어가는 것은 logits이지 확률이 아님!
optm = tf.train.AdamOptimizer(learning_rate = 0.001).minimize(cost)
corr = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accr= tf.reduce_mean(tf.cast(corr,"float"))

# INITIALIZER
init = tf.global_variables_initializer()
print ("FUNCTIONS READY")

FUNCTIONS READY


In [12]:
# PARAMETERS
training_epochs = 20
batch_size      = 100
display_step    = 4
# LAUNCH THE GRAPH
sess = tf.Session()
sess.run(init)
# OPTIMIZE
for epoch in range(training_epochs):
    avg_cost = 0.
    total_batch = int(mnist.train.num_examples/batch_size)
    # ITERATION
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feeds = {x: batch_xs, y: batch_ys}
        sess.run(optm, feed_dict=feeds)
        avg_cost += sess.run(cost, feed_dict=feeds)
    avg_cost = avg_cost / total_batch
    # DISPLAY
    if (epoch+1) % display_step == 0:
        print ("Epoch: %03d/%03d cost: %.9f" % (epoch, training_epochs, avg_cost))
        feeds = {x: batch_xs, y: batch_ys}
        train_acc = sess.run(accr, feed_dict=feeds)
        print ("TRAIN ACCURACY: %.3f" % (train_acc))
        feeds = {x: mnist.test.images, y: mnist.test.labels}
        test_acc = sess.run(accr, feed_dict=feeds)
        print ("TEST ACCURACY: %.3f" % (test_acc))
print ("OPTIMIZATION FINISHED")

Epoch: 003/020 cost: 0.123953741
TRAIN ACCURACY: 0.970
TEST ACCURACY: 0.964
Epoch: 007/020 cost: 0.048301718
TRAIN ACCURACY: 0.980
TEST ACCURACY: 0.975
Epoch: 011/020 cost: 0.019357661
TRAIN ACCURACY: 0.990
TEST ACCURACY: 0.980
Epoch: 015/020 cost: 0.007246731
TRAIN ACCURACY: 1.000
TEST ACCURACY: 0.980
Epoch: 019/020 cost: 0.003026320
TRAIN ACCURACY: 0.990
TEST ACCURACY: 0.980
OPTIMIZATION FINISHED
